In [ ]:
library(statsecol)
library(jagsUI)
library(MCMCvis)
library(ggplot2)
data("voles")

In [ ]:
sink("voles.txt")
cat("
model{
  # Prior specification
  psi1 ~ dunif(0, 1)
  mu ~ dnorm(0, 0.001)
  sigma ~ dunif(0, 100)
  b0 ~ dnorm(0, 0.001)
  b1 ~ dnorm(0, 0.001)
  a0 ~ dnorm(0, 0.001)
  a1 ~ dnorm(0, 0.001)

  for(site in 1:n.sites) {
    for(time in 1:(n.years - 1)) {
      logit(gamma[site]) = b0 + b1 * connectivity[site]      
      logit(epsilon[site]) = a0 + a1 * length[site]
    }
  }
   
  # Likelihood
  for(site in 1:n.sites){
    z[site, 1] ~ dbern(psi1)
    for(year in 2:n.years) {      
      z[site, year] ~ dbern(
        (1 - z[site, year - 1]) * gamma[site, year - 1] + z[site, year - 1] * (1 - epsilon[site, year - 1])
      )
    }
  }

  for(site in 1:n.sites) {
    for(year in 1:n.years) {
      q[site, year] ~ dnorm(mu, sigma^(-2))
      logit(p[site, year]) = q[site, year]
      y[site, year] ~ dbinom(p[site, year] * z[site, year], visits[site, year])

      ysim[site, year] ~ dbin(p[site, year] * z[site, year], visits[site, year])
      yexp[site, year] <- p[site, year] * visits[site, year] * z[site, year] + 0.001
      x2.obs[site, year] <- pow((y[site, year] - yexp[site, year]), 2) / (yexp[site, year])    # for observed data
      x2.sim[site, year] <- pow((ysim[site, year] - yexp[site, year]), 2) / (yexp[site, year]) # for 'ideal' data
      ft.obs[site, year] <- pow(sqrt(y[site, year]) - sqrt(yexp[site, year]), 2)               # for observed data
      ft.sim[site, year] <- pow(sqrt(ysim[site, year]) - sqrt(yexp[site, year]), 2)            # for 'ideal' data
    }
  }

  Chi2.obs <- sum(x2.obs[,])
  Chi2.sim <- sum(x2.sim[,])
  Chi2.ratio <- x2.obs/x2.sim
  FT.obs <- sum(ft.obs[,])
  FT.sim <- sum(ft.sim[,])
  FT.ratio <- FT.obs/FT.sim

  for(year in 1:n.years){
    propocc[year] <- sum(z[, year]) / n.sites   
  }
}
",fill = TRUE)
sink()

In [ ]:
visits = as.matrix(voles[,c("j1", "j2", "j3", "j4")])
obs = as.matrix(voles[, c("y1", "y2", "y3", "y4")])

In [ ]:
# Bundle data
volesdata <- list(
    y = obs,
    n.years = 4,
    n.sites = nrow(voles),
    visits = visits,
    connectivity = voles$Connectivity,
    length = voles$Length
)

volesinits <- function() {
  list(psi1 = runif(1, 0, 1),
       mu = runif(1, 0, 5),
       sigma = runif(1, 0, 1),
       b0 = runif(1, -2, 2),
       b1 = runif(1, -2, 2),
       a0 = runif(1, -2, 2),
       a1 = runif(1, -2, 2),
       z = ifelse(obs>0,1,0))
}

# Parameters monitored
volesparams <- c("gamma", "epsilon", "p", "propocc", 
                 "Chi2.obs", "Chi2.sim", "ft.obs", "ft.sim")

# MCMC settings
ni <- 10000
nt <- 4
nb <- 5000
nc <- 3

volesout <- jags(
    data = volesdata,
    inits = volesinits,
    parameters.to.save = volesparams,
    model.file = "voles.txt",
    n.chains = nc,
    n.iter = ni,
    n.burnin = nb,
    n.thin = nt
)

In [ ]:
MCMCtrace(volesout,                 #the fitted model
          params = volesparams[1:3],   #core model parameters
          iter = ni,                 #plot all iterations
          pdf = FALSE,               #DON'T write to a PDF
          type = "trace")

In [ ]:
burnet_dyn <- data.frame(Year = c(1:4,seq(1.5,5.5,1),seq(1.5,5.5,1)),
                         Parameter = c(rep("Occupancy",6),
                                       rep("Colonisation",5),
                                       rep("Extinction",5)),
                         Estimate = c(volesout$mean$propocc,
                                      volesout$mean$gamma,
                                      volesout$mean$epsilon),
                         Lower = c(volesout$q2.5$propocc,
                                   volesout$q2.5$gamma,
                                   volesout$q2.5$epsilon),
                         Upper = c(volesout$q97.5$propocc,
                                   volesout$q97.5$gamma,
                                   volesout$q97.5$epsilon))

ggplot(data = burnet_dyn, aes(x=Year,y=Estimate, color=Parameter)) + 
  geom_errorbar(aes(ymin=Lower, ymax=Upper), width=0) +
  geom_point(size=3) +
  facet_wrap(.~Parameter) +
  theme_bw()

In [ ]:
burnet_dyn